In [ ]:
%reload_ext autoreload
%autoreload 2
import sympy as sp
sp.init_printing()

## Continuous Weighted Enhanced Random Graph model
This models constraints both the number of links and total weight by the Lagrange multipliers $\alpha$ and $\beta$
The Hamiltonian of the problem is hence $H=\sum_{i<j} \alpha \Theta(w_{ij}-t) + \beta w_{ij}\Theta(w_{ij}-t)$.
We declare all the symbols necessary to continue.

In [ ]:
alpha = sp.Symbol('\\alpha',real=True,positive=True)
x = sp.Symbol('x',real=True,positive=True)

beta = sp.Symbol('\\beta',real=True,positive=True)
y = sp.Symbol('y',real=True,positive=True)

wij = sp.Symbol('w_{ij}',real=True,positive=True)
t = sp.Symbol('t',real=True,positive=True)

H = alpha * sp.Heaviside(wij-t) + beta * wij * sp.Heaviside(wij-t)

Now we perform the calculation of the partition function $Z=\int_{0}^{\infty} e^{-H} dw$ with the result $$Z=t + \frac{e^{-\alpha-t\beta}}{\beta}$$

In [ ]:
Z = sp.integrate(sp.exp(-H).rewrite(sp.Piecewise),(wij,0,sp.oo))
Z

Calculation of ensemble quantities
----------------------------------
Compute the free energy $F=-\log(Z)$. 
By taking the derivatives $\frac{\partial F}{\partial \alpha_{ij}}$, $\frac{\partial F}{\partial \alpha_{ij}}$ we get the expected link probability $\langle a_{ij} \rangle$ and the expected link weight $\langle w_{ij} \rangle$ respectively:

In [ ]:
F = -sp.log(Z)
dFdalpha = sp.diff(F,alpha)
dFdbeta = sp.diff(F,beta)

However we need to use the variables $x_i,x_j$ and $y_i,y_j$ for the Lagrange multipliers, for notational convention. They are obtained as $x_i = e^{-\alpha_i}$ and $y_i=e^{-\beta_i}$, so we create some replacements dictionaries `repl,repl2,repl3`

In [ ]:
repl = {alpha: -sp.log(x), beta: -sp.log(y) }

Now we obtain an expression for $\langle a_{ij} \rangle$

In [ ]:
expected_aij = sp.simplify(dFdalpha.subs(repl))
expected_aij

and proceed similarly for the expected link weights $\langle w_{ij} \rangle$

In [ ]:
expected_wij = sp.simplify(dFdbeta.subs(repl))
expected_wij

From this last formula we find that the expected link weight is obtained from the link probability multiplied by a factor $\frac{\log{\left (\left(y_{i} y_{j}\right)^{t} \right )} - 1}{\log{\left (y_{i} y_{j} \right )}}$

In [ ]:
sp.simplify(expected_wij/expected_aij)

Here we also try to export the formulas as lambda expressions to include in Python directly

The graph probability $P(G)=e^{-H}/Z$ and the loglikelihood is $\log P$

In [ ]:
P = sp.simplify(sp.exp(-H)/Z)
logL = sp.log(sp.expand_log(sp.simplify(P.subs(repl))))
sp.expand_log(logL)

In [ ]:
from sympy.utilities.lambdify import lambdastr
lambdastr((xi,xj,yi,yj), sp.simplify((sp.diff(pij.subs(repl2),betai)).subs(repl3)))

In [ ]:
expected_aij = sp.simplify(sp.expand_log(pij.subs(repl)))
expected_wij = sp.simplify(sp.expand_log(wij.subs(repl)))

Constraining only strength sequence
-----------------------------------

Now let us study a model where only the strength sequence is constrained $s_i = \sum_j w_{ij}\Theta(w_{ij}-t)$. To do this we must repeat all the previous calculations in the settings $\alpha_{ij}=0$, or equivalently with an Hamiltonian $$H=\sum_{i<j} \beta_{ij}w_{ij}\Theta(w_{ij}-t)$$

In [ ]:
H = betaij * wij * sp.Heaviside(wij-t)

and compute the partition function $Z$

In [ ]:
Z = sp.integrate(sp.exp(-H).rewrite(sp.Piecewise),(wij,0,sp.oo))
Z

the free energy $F=-\log(Z)$ reads like

In [ ]:
F = sp.simplify(-sp.log(Z))
F

The expected link probabability is $\langle w_{ij} \rangle$

The graph probability $P(W)=e^{-H}/Z$ and the likelihood reads like

In [ ]:
P=sp.simplify((sp.exp(-H)/Z))
logL = sp.simplify(sp.expand_log(sp.log(P)))
logL

In [ ]:
import sys
sys.path.append('/home/carlo2/workspace/networkqit/')
import networkqit as nq

nq.CWTCM(N)